# Download three Polish models from the Huggingface repository

In [2]:
from transformers import pipeline

2022-11-25 08:46:25.965277: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
unmasker_roberta = pipeline("fill-mask", model="xlm-roberta-base")
unmasker_bert = pipeline('fill-mask', model='bert-base-multilingual-cased')
unmasker_distilbert = pipeline('fill-mask', model='distilbert-base-multilingual-cased')

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/542M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

# Devise a method to test if the langage model understands Polish cases

In [5]:
import pandas as pd

In [32]:
def test_models(s):
    index = pd.MultiIndex.from_tuples([
        ('roberta', 'score'),
        ('roberta', 'token'),
        ('roberta', 'sentence'),
        ('bert', 'score'),
        ('bert', 'token'),
        ('bert', 'sentence'),
        ('distilbert', 'score'),
        ('distilbert', 'token'),
        ('distilbert', 'sentence'),
    ], names=['model', 'field'])
    df_roberta = pd.DataFrame(
        [(entry['score'], entry['token_str'], entry['sequence']) for entry in unmasker_roberta(s.replace('[MASK]', '<mask>'))],
        columns=['score', 'token', 'sequence']
    )
    df_bert = pd.DataFrame(
        [(entry['score'], entry['token_str'], entry['sequence']) for entry in unmasker_bert(s)],
        columns=['score', 'token', 'sequence']
    )
    df_distilbert = pd.DataFrame(
        [(entry['score'], entry['token_str'], entry['sequence']) for entry in unmasker_distilbert(s)],
        columns=['score', 'token', 'sequence']
    )
    df = pd.concat([df_roberta, df_bert, df_distilbert], axis=1)
    df.columns = index
    return df

### Mianownik

In [36]:
test_models("Warszawa to największe [MASK].")

model   roberta                                              bert            \
field     score   token                        sentence     score     token   
0      0.489236  miasto  Warszawa to największe miasto.  0.271117    miasto   
1      0.045349  miasta  Warszawa to największe miasta.  0.088363    miasta   
2      0.028841                Warszawa to największe .  0.057303       woj   
3      0.019006  Miasto  Warszawa to największe Miasto.  0.036420  Warszawa   
4      0.018972       i       Warszawa to największe i.  0.035606        to   

model                                   distilbert             \
field                          sentence      score      token   
0        Warszawa to największe miasto.   0.649291     miasto   
1        Warszawa to największe miasta.   0.104943     miasta   
2           Warszawa to największe woj.   0.034653     Miasto   
3      Warszawa to największe Warszawa.   0.018810    centrum   
4            Warszawa to największe to.   0.013046  dzielnicy   

model                                     
field                           sentence  
0         Warszawa to największe miasto.  
1         Warszawa to największe miasta.  
2         Warszawa to największe Miasto.  
3        Warszawa to największe centrum.  
4      Warszawa to największe dzielnicy.

### Dopełniacz

In [37]:
test_models("Maciek nie zjadł [MASK].")

model   roberta                                            bert        \
field     score     token                    sentence     score token   
0      0.161912     mleka     Maciek nie zjadł mleka.  0.100141   się   
1      0.071890         .          Maciek nie zjadł..  0.063438    go   
2      0.060829      wody      Maciek nie zjadł wody.  0.016943  tego   
3      0.050582       się       Maciek nie zjadł się.  0.014984    mu   
4      0.042187  alkoholu  Maciek nie zjadł alkoholu.  0.013580   nie   

model                         distilbert                                    
field                sentence      score   token                  sentence  
0       Maciek nie zjadł się.   0.193708  короля  Maciek nie zjadł короля.  
1        Maciek nie zjadł go.   0.027688      go      Maciek nie zjadł go.  
2      Maciek nie zjadł tego.   0.023025     nic     Maciek nie zjadł nic.  
3        Maciek nie zjadł mu.   0.022320       缚       Maciek nie zjadł 缚.  
4       Maciek nie zjadł nie.   0.021823   trees   Maciek nie zjadł trees.

### Celownik

In [38]:
test_models("Zwierzyłem się [MASK] ze swoich wątpliwości.")

model   roberta                                                          \
field     score    token                                       sentence   
0      0.533098   jednej   Zwierzyłem się jednej ze swoich wątpliwości.   
1      0.212774   jednym   Zwierzyłem się jednym ze swoich wątpliwości.   
2      0.043181  każdemu  Zwierzyłem się każdemu ze swoich wątpliwości.   
3      0.025095    jedną    Zwierzyłem się jedną ze swoich wątpliwości.   
4      0.016671  jednego  Zwierzyłem się jednego ze swoich wątpliwości.   

model      bert                                                              \
field     score      token                                         sentence   
0      0.056296     jednym     Zwierzyłem się jednym ze swoich wątpliwości.   
1      0.049705     dobrze     Zwierzyłem się dobrze ze swoich wątpliwości.   
2      0.049539      tylko      Zwierzyłem się tylko ze swoich wątpliwości.   
3      0.036629         to         Zwierzyłem się to ze swoich wątpliwości.   
4      0.035478  wszystkim  Zwierzyłem się wszystkim ze swoich wątpliwości.   

model distilbert                                                              
field      score      token                                         sentence  
0       0.126141  wszystkim  Zwierzyłem się wszystkim ze swoich wątpliwości.  
1       0.071580     jednym     Zwierzyłem się jednym ze swoich wątpliwości.  
2       0.061009      tylko      Zwierzyłem się tylko ze swoich wątpliwości.  
3       0.040028      jeden      Zwierzyłem się jeden ze swoich wątpliwości.  
4       0.030120      znany      Zwierzyłem się znany ze swoich wątpliwości.

### Biernik

In [39]:
test_models("Czekam na [MASK].")

model   roberta                                 bert                        \
field     score   token           sentence     score token        sentence   
0      0.130356     Vas     Czekam na Vas.  0.021675    to   Czekam na to.   
1      0.086805     vas     Czekam na vas.  0.016796   ##ł     Czekam nał.   
2      0.046575     Vás     Czekam na Vás.  0.015543   rok  Czekam na rok.   
3      0.038227  Ciebie  Czekam na Ciebie.  0.014566    op   Czekam na op.   
4      0.037189    mail    Czekam na mail.  0.013793   wyd  Czekam na wyd.   

model distilbert                                   
field      score      token              sentence  
0       0.094500      ziemi      Czekam na ziemi.  
1       0.030968      Ziemi      Czekam na Ziemi.  
2       0.022952   południe   Czekam na południe.  
3       0.019232  Slovensku  Czekam na Slovensku.  
4       0.018215        cm³        Czekam na cm³.

### Narzędnik

In [41]:
test_models("Interesuję się [MASK].")

model   roberta                                    bert        \
field     score  token               sentence     score token   
0      0.147862    tym    Interesuję się tym.  0.121616    pt   
1      0.041448    nią    Interesuję się nią.  0.038328    np   
2      0.035677  osobą  Interesuję się osobą.  0.034657    in   
3      0.033390      .       Interesuję się..  0.031437    ks   
4      0.031919    nim    Interesuję się nim.  0.028935     p   

model                     distilbert                                          
field            sentence      score       token                    sentence  
0      Interesuję się pt.   0.063216         tzw         Interesuję się tzw.  
1      Interesuję się np.   0.055185  dwukrotnie  Interesuję się dwukrotnie.  
2      Interesuję się in.   0.022601          np          Interesuję się np.  
3      Interesuję się ks.   0.016866         tez         Interesuję się tez.  
4       Interesuję się p.   0.015592       wtedy       Interesuję się wtedy.

### Miejscownik

In [42]:
test_models("Twoje dzieci siedzą na [MASK].")

model   roberta                                            bert           \
field     score  token                       sentence     score    token   
0      0.574236  stole  Twoje dzieci siedzą na stole.  0.077923      tzw   
1      0.036548     tv     Twoje dzieci siedzą na tv.  0.068905   wyspie   
2      0.024711     TV     Twoje dzieci siedzą na TV.  0.038716  miejscu   
3      0.022114   sofa   Twoje dzieci siedzą na sofa.  0.037536    ziemi   
4      0.020351   sali   Twoje dzieci siedzą na sali.  0.023147     polu   

model                                  distilbert             \
field                         sentence      score      token   
0          Twoje dzieci siedzą na tzw.   0.085258      ziemi   
1       Twoje dzieci siedzą na wyspie.   0.085206        cm³   
2      Twoje dzieci siedzą na miejscu.   0.041236      ulicy   
3        Twoje dzieci siedzą na ziemi.   0.033868      Ziemi   
4         Twoje dzieci siedzą na polu.   0.029422  ##damente   

model                                  
field                        sentence  
0       Twoje dzieci siedzą na ziemi.  
1         Twoje dzieci siedzą na cm³.  
2       Twoje dzieci siedzą na ulicy.  
3       Twoje dzieci siedzą na Ziemi.  
4      Twoje dzieci siedzą nadamente.

### Wołacz

In [61]:
test_models("Panie [MASK], co mi dolega?")

model   roberta                                           bert        \
field     score   token                     sentence     score token   
0      0.053629       ,        Panie,, co mi dolega?  0.039060  ##ña   
1      0.044342       a        Paniea, co mi dolega?  0.034079   pan   
2      0.023499   Maria   Panie Maria, co mi dolega?  0.022050     o   
3      0.020907  doktor  Panie doktor, co mi dolega?  0.019305   Pan   
4      0.018776       e        Paniee, co mi dolega?  0.018763   ##a   

model                           distilbert                                  
field                  sentence      score token                  sentence  
0        Panieña, co mi dolega?   0.024658   pan  Panie pan, co mi dolega?  
1      Panie pan, co mi dolega?   0.021898     !     Panie!, co mi dolega?  
2        Panie o, co mi dolega?   0.021284     ?     Panie?, co mi dolega?  
3      Panie Pan, co mi dolega?   0.011152  ##go    Paniego, co mi dolega?  
4         Paniea, co mi dolega?   0.011095   ##w     Paniew, co mi dolega?

# Devise a method to test long-range relationships such as gender

In [53]:
test_models("Skończyłam pracę i [MASK] do domu.")

model   roberta                                                        bert  \
field     score      token                               sentence     score   
0      0.266778      wraca      Skończyłam pracę i wraca do domu.  0.054037   
1      0.132109      wróci      Skończyłam pracę i wróci do domu.  0.031863   
2      0.097389        mam        Skończyłam pracę i mam do domu.  0.029584   
3      0.080546  zapraszam  Skończyłam pracę i zapraszam do domu.  0.024074   
4      0.047741     jestem     Skończyłam pracę i jestem do domu.  0.023573   

model                                             distilbert             \
field   token                            sentence      score      token   
0      wrócił  Skończyłam pracę i wrócił do domu.   0.122379   pracował   
1        domu    Skończyłam pracę i domu do domu.   0.097319      školu   
2         mam     Skończyłam pracę i mam do domu.   0.041374      kishi   
3         kam     Skończyłam pracę i kam do domu.   0.039966  prowadził   
4         tam     Skończyłam pracę i tam do domu.   0.038897      pracy   

model                                         
field                               sentence  
0       Skończyłam pracę i pracował do domu.  
1          Skończyłam pracę i školu do domu.  
2          Skończyłam pracę i kishi do domu.  
3      Skończyłam pracę i prowadził do domu.  
4          Skończyłam pracę i pracy do domu.

In [51]:
test_models("Oglądałem telewizję i [MASK] obiad.")

model   roberta                                                       bert  \
field     score     token                               sentence     score   
0      0.217025  mieliśmy  Oglądałem telewizję i mieliśmy obiad.  0.142903   
1      0.202638    miałem    Oglądałem telewizję i miałem obiad.  0.073793   
2      0.140416      dali      Oglądałem telewizję i dali obiad.  0.066067   
3      0.065819        na        Oglądałem telewizję i na obiad.  0.038057   
4      0.042579    zrobił    Oglądałem telewizję i zrobił obiad.  0.036171   

model                                            distilbert             \
field  token                            sentence      score      token   
0        sam    Oglądałem telewizję i sam obiad.   0.227742  prowadził   
1        ten    Oglądałem telewizję i ten obiad.   0.101742    autorem   
2        ich    Oglądałem telewizję i ich obiad.   0.045809       jego   
3         to     Oglądałem telewizję i to obiad.   0.029787   prowadzi   
4      każdy  Oglądałem telewizję i każdy obiad.   0.021066      wielu   

model                                          
field                                sentence  
0      Oglądałem telewizję i prowadził obiad.  
1        Oglądałem telewizję i autorem obiad.  
2           Oglądałem telewizję i jego obiad.  
3       Oglądałem telewizję i prowadzi obiad.  
4          Oglądałem telewizję i wielu obiad.

In [52]:
test_models("Grałem w piłkę i [MASK] gola.")

model   roberta                                             bert            \
field     score   token                       sentence     score     token   
0      0.364927  miałem  Grałem w piłkę i miałem gola.  0.481213    zdobył   
1      0.095586     mam     Grałem w piłkę i mam gola.  0.403535  strzelił   
2      0.095476  zdobył  Grałem w piłkę i zdobył gola.  0.038899      miał   
3      0.046102      na      Grałem w piłkę i na gola.  0.011557       bez   
4      0.034628  dostał  Grałem w piłkę i dostał gola.  0.003847      było   

model                                  distilbert            \
field                         sentence      score     token   
0        Grałem w piłkę i zdobył gola.   0.519943    zdobył   
1      Grałem w piłkę i strzelił gola.   0.092554   jednego   
2          Grałem w piłkę i miał gola.   0.044560      miał   
3           Grałem w piłkę i bez gola.   0.021202  strzelił   
4          Grałem w piłkę i było gola.   0.020905       dwa   

model                                   
field                         sentence  
0        Grałem w piłkę i zdobył gola.  
1       Grałem w piłkę i jednego gola.  
2          Grałem w piłkę i miał gola.  
3      Grałem w piłkę i strzelił gola.  
4           Grałem w piłkę i dwa gola.

# Check if the model captures real-world knolwedge

In [62]:
test_models("[MASK] wrze w temperaturze stu stopni, a zamarza w temperaturze zera stopni Celsjusza.")

model   roberta               \
field     score        token   
0      0.044129        Dzień   
1      0.033657        Kiedy   
2      0.030155  Najczęściej   
3      0.025810        Teraz   
4      0.025090          Gdy   

model                                                         bert        \
field                                           sentence     score token   
0      Dzień wrze w temperaturze stu stopni, a zamarz...  0.044685    Za   
1      Kiedy wrze w temperaturze stu stopni, a zamarz...  0.039543    Po   
2      Najczęściej wrze w temperaturze stu stopni, a ...  0.029791   Nie   
3      Teraz wrze w temperaturze stu stopni, a zamarz...  0.026823     A   
4      Gdy wrze w temperaturze stu stopni, a zamarza ...  0.022241     W   

model                                                    distilbert        \
field                                           sentence      score token   
0      Za wrze w temperaturze stu stopni, a zamarza w...   0.078937    Na   
1      Po wrze w temperaturze stu stopni, a zamarza w...   0.050302    We   
2      Nie wrze w temperaturze stu stopni, a zamarza ...   0.040866     W   
3      A wrze w temperaturze stu stopni, a zamarza w ...   0.025502    Od   
4      W wrze w temperaturze stu stopni, a zamarza w ...   0.024814    Za   

model                                                     
field                                           sentence  
0      Na wrze w temperaturze stu stopni, a zamarza w...  
1      We wrze w temperaturze stu stopni, a zamarza w...  
2      W wrze w temperaturze stu stopni, a zamarza w ...  
3      Od wrze w temperaturze stu stopni, a zamarza w...  
4      Za wrze w temperaturze stu stopni, a zamarza w...

In [60]:
test_models("Stolicą Niemiec jest [MASK].")

model   roberta                                                bert  \
field     score     token                        sentence     score   
0      0.368489    Polska    Stolicą Niemiec jest Polska.  0.151338   
1      0.117036  Warszawa  Stolicą Niemiec jest Warszawa.  0.110658   
2      0.104238    Berlin    Stolicą Niemiec jest Berlin.  0.048058   
3      0.025937    Europa    Stolicą Niemiec jest Europa.  0.040851   
4      0.021986    Bayern    Stolicą Niemiec jest Bayern.  0.035701   

model                                             distilbert           \
field      token                         sentence      score    token   
0              r          Stolicą Niemiec jest r.   0.119010        핫   
1        Niemiec    Stolicą Niemiec jest Niemiec.   0.045764   Niemcy   
2         Weimar     Stolicą Niemiec jest Weimar.   0.037942  Niemiec   
3             ok         Stolicą Niemiec jest ok.   0.036939  Wrocław   
4      niemiecki  Stolicą Niemiec jest niemiecki.   0.036267     няма   

model                                 
field                       sentence  
0            Stolicą Niemiec jest 핫.  
1       Stolicą Niemiec jest Niemcy.  
2      Stolicą Niemiec jest Niemiec.  
3      Stolicą Niemiec jest Wrocław.  
4         Stolicą Niemiec jest няма.

In [59]:
test_models("Trawa jest koloru [MASK].")

model   roberta                                             bert        \
field     score     token                     sentence     score token   
0      0.134585         .          Trawa jest koloru..  0.039318    mm   
1      0.031441  czerwony  Trawa jest koloru czerwony.  0.028913     K   
2      0.031084         ,          Trawa jest koloru,.  0.016017     r   
3      0.028529         :          Trawa jest koloru:.  0.011481     .   
4      0.026251    drewna    Trawa jest koloru drewna.  0.011459   bar   

model                         distilbert                                   
field                sentence      score  token                  sentence  
0       Trawa jest koloru mm.   0.045270      菊      Trawa jest koloru 菊.  
1        Trawa jest koloru K.   0.036054  temas  Trawa jest koloru temas.  
2        Trawa jest koloru r.   0.031620      駕      Trawa jest koloru 駕.  
3         Trawa jest koloru..   0.024632  ##won     Trawa jest koloruwon.  
4      Trawa jest koloru bar.   0.020583      瑕      Trawa jest koloru 瑕.

# 